In [1]:
import cv2
import numpy as np
import pandas as pd
import pickle
import xgboost as xgb
from matplotlib import pyplot as plt
import skimage
import os
import sys
import sqlite3
import shutil
from time import time
sys.path.append(os.environ['REPO_DIR'])
from lib.utils import configuration, run
from matplotlib.path import Path
from shapely.geometry import Polygon

In [2]:
def CDF(x):
    x=np.sort(x)
    size=x.shape[0]
    y=np.arange(0,size)/size
    return x,y


def setup_download_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if os.path.exists(local_fp):
        print('ALREADY DOWNLOADED FILE')
        return

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(s3_fp, local_fp))
    else:
        run('aws s3 cp {0} {1}'.format(s3_fp, local_fp))

def setup_upload_from_s3(rel_fp, recursive=True):
    s3_fp = 's3://mousebrainatlas-data/' + rel_fp
    local_fp = os.environ['ROOT_DIR'] + rel_fp

    if recursive:
        run('aws s3 cp --recursive {0} {1}'.format(local_fp, s3_fp))
    else:
        run('aws s3 cp {0} {1}'.format(local_fp, s3_fp))

In [3]:
stack = 'MD594'
fp = os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt')
setup_download_from_s3(fp, recursive=False)
with open(os.environ['ROOT_DIR']+fp, 'r') as f:
    fn_idx_tuples = [line.strip().split() for line in f.readlines()]
    section_to_filename = {int(idx): fn for fn, idx in fn_idx_tuples}


fname = os.path.join('CSHL_data_processed', stack, 'Annotation.npy')
setup_download_from_s3(fname, recursive=False)
annotation = np.load(os.environ['ROOT_DIR']+fname, allow_pickle = True, encoding='latin1')
contours = pd.DataFrame(annotation)
contours = contours.rename(columns={0:"name", 1:"section", 2:"vertices"})
contours_grouped = contours.groupby('section')
valid_sections = np.sort(contours['section'].unique())

fn = 'CSHL_data_processed/MD589/ThresholdsV2.pkl'
setup_download_from_s3(fn, recursive=False)
thresholds = pickle.load(open(os.environ['ROOT_DIR']+fn,'rb'))

ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE


In [7]:
savepath = 'CSHL_cells_mark/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+stack+'/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)
savepath = savepath+'color/'
if not os.path.exists(os.environ['ROOT_DIR']+savepath):
    os.mkdir(os.environ['ROOT_DIR']+savepath)

resol = 0.46

paired_structures = ['5N', '6N', '7N', '7n', 'Amb', 'LC', 'LRt', 'Pn', 'Tz', 'VLL', 'RMC', \
                     'SNC', 'SNR', '3N', '4N', 'Sp5I', 'Sp5O', 'Sp5C', 'PBG', '10N', 'VCA', 'VCP', 'DC']
singular_structures = ['AP', '12N', 'RtTg', 'SC', 'IC']

all_structures = paired_structures + singular_structures

margin = 500/0.46

In [6]:
len(valid_sections)

265

In [8]:
%%time
raw_images_root = 'CSHL_data_processed/'+stack+'/'+stack+'_prep2_lossless_gray/'
for section in valid_sections:
    img_fn = raw_images_root + section_to_filename[section] + '_prep2_lossless_gray.tif'
#     setup_download_from_s3(img_fn, recursive=False)
    img = cv2.imread(os.environ['ROOT_DIR']+img_fn, 2)
    whole = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    polygons = [(contour['name'], contour['vertices']) \
            for contour_id, contour in contours_grouped.get_group(section).iterrows()]
    count = 0    
    cs = []
    for contour_id, contour in polygons:
        structure = contour_id
        if structure not in all_structures:
            continue
        polygon = contour.copy()
        cs.append(polygon.astype(np.int32))
        count += 1
        print(section, structure, count, '/', len(polygons))
    com = cv2.polylines(whole.copy(), cs, True, [255, 0, 0], 3, lineType=50)
    filename = savepath + str(section) + '.tif'
    com = cv2.cvtColor(com, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.environ['ROOT_DIR']+filename, com)    

94 VCA 1 / 1
95 VCA 1 / 1
96 VCA 1 / 1
97 VCA 1 / 1
98 VCA 1 / 1
99 VCA 1 / 1
100 VCA 1 / 2
101 VCA 1 / 1
102 VCA 1 / 1
103 VCA 1 / 1
104 VCA 1 / 1
105 VCA 1 / 3
105 DC 2 / 3
105 VCP 3 / 3
106 VCP 1 / 3
106 DC 2 / 3
106 VCA 3 / 3
107 VCA 1 / 3
107 DC 2 / 3
107 VCP 3 / 3
108 VCP 1 / 3
108 VCA 2 / 3
108 DC 3 / 3
109 VCP 1 / 3
109 VCA 2 / 3
109 DC 3 / 3
110 VCA 1 / 3
110 VCP 2 / 3
110 DC 3 / 3
111 VCA 1 / 3
111 DC 2 / 3
111 VCP 3 / 3
112 DC 1 / 3
112 VCP 2 / 3
112 VCA 3 / 3
113 VCP 1 / 3
113 DC 2 / 3
113 VCA 3 / 3
114 DC 1 / 3
114 VCA 2 / 3
114 VCP 3 / 3
115 VCA 1 / 3
115 VCP 2 / 3
115 DC 3 / 3
116 VCP 1 / 3
116 DC 2 / 3
116 VCA 3 / 3
117 DC 1 / 3
117 VCP 2 / 3
117 VCA 3 / 3
118 DC 1 / 3
118 VCA 2 / 3
118 VCP 3 / 3
119 DC 1 / 3
119 VCA 2 / 3
119 VCP 3 / 3
120 VCA 1 / 4
120 VCP 2 / 4
120 DC 3 / 4
121 VCA 1 / 4
121 VCP 2 / 4
121 DC 3 / 4
122 DC 1 / 4
122 VCA 2 / 4
123 DC 1 / 4
123 VCA 2 / 4
124 DC 1 / 4
124 VCA 2 / 4
125 SNR 1 / 4
125 DC 2 / 4
126 SNR 1 / 6
126 DC 2 / 6
126 Sp5O 3 / 6
126 S

181 Pn 1 / 10
181 SC 2 / 10
181 SNR 3 / 10
181 LRt 4 / 10
181 7N 5 / 10
181 Sp5C 6 / 10
181 IC 7 / 10
181 7n 8 / 10
181 LC 9 / 10
181 SNC 10 / 10
182 Sp5C 1 / 10
182 LRt 2 / 10
182 7n 3 / 10
182 7N 4 / 10
182 SNC 5 / 10
182 SC 6 / 10
182 Pn 7 / 10
182 LC 8 / 10
182 SNR 9 / 10
182 IC 10 / 10
183 Sp5C 1 / 10
183 Pn 2 / 10
183 SNR 3 / 10
183 SNC 4 / 10
183 IC 5 / 10
183 7N 6 / 10
183 LRt 7 / 10
183 7n 8 / 10
183 LC 9 / 10
183 SC 10 / 10
184 LC 1 / 11
184 SNR 2 / 11
184 SNC 3 / 11
184 Sp5C 4 / 11
184 7N 5 / 11
184 SC 6 / 11
184 IC 7 / 11
184 Pn 8 / 11
184 RMC 9 / 11
184 LRt 10 / 11
184 7n 11 / 11
185 Pn 1 / 11
185 SNC 2 / 11
185 7n 3 / 11
185 SNR 4 / 11
185 Sp5C 5 / 11
185 LRt 6 / 11
185 RMC 7 / 11
185 SC 8 / 11
185 LC 9 / 11
185 7N 10 / 11
185 IC 11 / 11
186 SC 1 / 11
186 IC 2 / 11
186 Pn 3 / 11
186 SNR 4 / 11
186 Sp5C 5 / 11
186 LC 6 / 11
186 LRt 7 / 11
186 RMC 8 / 11
186 SNC 9 / 11
186 7n 10 / 11
186 7N 11 / 11
188 SC 1 / 13
188 Pn 2 / 13
188 7n 3 / 13
188 LRt 4 / 13
188 7N 5 / 13
188 R

253 10N 1 / 9
253 RMC 2 / 9
253 IC 3 / 9
253 RtTg 4 / 9
253 Tz 5 / 9
253 7n 6 / 9
253 Pn 7 / 9
253 SC 8 / 9
254 RtTg 1 / 13
254 10N 2 / 13
254 IC 3 / 13
254 7n 4 / 13
254 SNR 5 / 13
254 SC 6 / 13
254 Pn 7 / 13
254 Tz 8 / 13
254 RMC 9 / 13
255 RMC 1 / 10
255 SNR 2 / 10
255 7n 3 / 10
255 10N 4 / 10
255 Tz 5 / 10
255 SC 6 / 10
255 IC 7 / 10
255 Pn 8 / 10
255 RtTg 9 / 10
256 Tz 1 / 9
256 RMC 2 / 9
256 RtTg 3 / 9
256 IC 4 / 9
256 SC 5 / 9
256 Pn 6 / 9
256 7n 7 / 9
256 SNR 8 / 9
257 IC 1 / 9
257 7n 2 / 9
257 RMC 3 / 9
257 RtTg 4 / 9
257 SNR 5 / 9
257 SC 6 / 9
257 Pn 7 / 9
257 Tz 8 / 9
258 7n 1 / 9
258 SNC 2 / 9
258 IC 3 / 9
258 RMC 4 / 9
258 Tz 5 / 9
258 RtTg 6 / 9
258 Pn 7 / 9
258 SC 8 / 9
259 RtTg 1 / 10
259 RMC 2 / 10
259 SNR 3 / 10
259 IC 4 / 10
259 SNC 5 / 10
259 Tz 6 / 10
259 SC 7 / 10
259 Pn 8 / 10
259 7n 9 / 10
260 Tz 1 / 10
260 SNR 2 / 10
260 SC 3 / 10
260 LC 4 / 10
260 Pn 5 / 10
260 7n 6 / 10
260 SNC 7 / 10
260 IC 8 / 10
260 RtTg 9 / 10
262 IC 1 / 10
262 Tz 2 / 10
262 7n 3 / 10
262

315 Sp5I 1 / 11
315 Sp5O 2 / 11
315 VLL 3 / 11
315 Sp5C 4 / 11
315 DC 5 / 11
315 PBG 6 / 11
315 SNR 7 / 11
316 DC 1 / 10
316 Sp5O 2 / 10
316 Sp5C 3 / 10
316 PBG 4 / 10
316 SNR 5 / 10
316 Sp5I 6 / 10
317 Sp5I 1 / 9
317 PBG 2 / 9
317 DC 3 / 9
317 Sp5O 4 / 9
317 Sp5C 5 / 9
318 Sp5O 1 / 9
318 Sp5I 2 / 9
318 PBG 3 / 9
318 DC 4 / 9
318 Sp5C 5 / 9
319 Sp5C 1 / 7
319 DC 2 / 7
319 Sp5O 3 / 7
319 Sp5I 4 / 7
320 DC 1 / 7
320 Sp5C 2 / 7
320 Sp5I 3 / 7
320 Sp5O 4 / 7
321 Sp5I 1 / 7
321 Sp5C 2 / 7
321 DC 3 / 7
321 Sp5O 4 / 7
322 Sp5I 1 / 7
322 DC 2 / 7
322 Sp5C 3 / 7
322 Sp5O 4 / 7
323 DC 1 / 7
323 Sp5I 2 / 7
323 Sp5O 3 / 7
323 Sp5C 4 / 7
324 Sp5O 1 / 7
324 Sp5I 2 / 7
324 Sp5C 3 / 7
324 DC 4 / 7
325 Sp5C 1 / 7
325 DC 2 / 7
325 Sp5I 3 / 7
325 Sp5O 4 / 7
326 DC 1 / 7
326 Sp5O 2 / 7
326 Sp5C 3 / 7
326 Sp5I 4 / 7
327 Sp5I 1 / 7
327 Sp5C 2 / 7
327 DC 3 / 7
327 Sp5O 4 / 7
328 VCA 1 / 6
328 Sp5I 2 / 6
328 Sp5O 3 / 6
328 Sp5C 4 / 6
328 DC 5 / 6
329 Sp5I 1 / 5
329 DC 2 / 5
329 Sp5O 3 / 5
329 VCA 4 / 5
330 Sp

In [10]:
script_dir = os.environ['REPO_DIR']
! python $script_dir/Cell_mark.py 'MD594' 94 

/Users/kuiqian/Github/venv/shapeology_venv/lib/python3.7/site-packages/sklearn/externals/joblib/externals/cloudpickle/cloudpickle.py:47: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
ALREADY DOWNLOADED FILE
run cmd= aws s3 cp /Users/kuiqian/BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/VCA/94_VCA_mark.png s3://mousebrainatlas-data/CSHL_cells_mark/MD594/VCA/94_VCA_mark.png
upload: ../../../BstemAtlasDataBackup/ucsd_brain/CSHL_cells_mark/MD594/VCA/94_VCA_mark.png to s3://mousebrainatlas-data/CSHL_cells_mark/MD594/VCA/94_VCA_mark.png
94 VCA 1 / 1
